# Unsupervisioned Learning (Aprendizado Não Supervisionado)

Prof. Daniel de Abreu Pereira Uhr

### Conteúdo
* Introdução
* Cross-Validation (Validação Cruzada)
  * The Validation Set Approach (A Abordagem do Conjunto de Validação)
  * Leave-One-Out Cross-Validation (Validação Cruzada Leave-One-Out)
  * k-Fold Cross-Validation (Validação Cruzada k-Fold)
* The Bootstrap

### Referências

* [An Introduction to Statistical Learning](https://www.statlearning.com/) (ISL) by Gareth James, Daniela Witten, Trevor Hastie and Robert Tibshirani
  * Capítulo 5***
* [The Elements of Statistical Learning](https://hastie.su.domains/ElemStatLearn/) (ESL) by Trevor Hastie, Robert Tibshirani and Jerome Friedman : 
  * Capítulo 7

***Disclaimer:*** *O material apresentado aqui é uma adaptação do material de aula do Prof. Daniel de Abreu Pereira Uhr, e não deve ser utilizado para fins comerciais. O material é disponibilizado para fins educacionais e de pesquisa, e não deve ser reproduzido sem a devida autorização do autor. Este material pode conter erros e imprecisões. O autor não se responsabiliza por quaisquer danos ou prejuízos decorrentes do uso deste material. O uso deste material é de responsabilidade exclusiva do usuário. Caso você encontre erros ou imprecisões neste material, por favor, entre em contato com o autor para que possam ser corrigidos. O autor agradece qualquer feedback ou sugestão de melhoria.*